# Customer Support Chatbot with Sentiment Analysis Functionality

This application combines AI and sentiment analysis tools to function as a customer service bot, allowing users to submit reviews and receive immediate assistance. It is particularly useful for builders and service providers with a large customer base, as it reduces the need to hire additional customer service agents, thereby lowering costs and increasing profits.

Tools Used:

1. Hugging Face Pipeline: Provides the sentiment analysis model to evaluate the customer’s review before passing it on to the AI.


2. Langchain: Serves as the AI platform for generating chatbot responses.


3. Python: The entire application is built using Python.


4. Gradio: Used to develop the chat interface.



Users can simply enter their review in the provided space and submit it using a dedicated button. The AI then generates a response within seconds, based on the sentiment analysis.

Tests:
* Positive sentiment test
![Positive sentiment test result](https://github.com/user-attachments/assets/3606c50b-5ccb-450f-9321-913a9e02c32d)

* Negative sentiment test
![Negative sentiment test result](https://github.com/user-attachments/assets/367b4d42-08e6-4f52-a087-8ba1755b594c)

In [13]:
%pip install --upgrade --quiet huggingface_hub langchain_huggingface langchain langchain_community langchain_google_genai gradio

In [14]:
import os
import gradio as gr
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from transformers import pipeline
from langchain_google_genai import GoogleGenerativeAI

In [16]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [17]:
GOOGLE_API_KEY = getpass()

··········


In [18]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [19]:
template = """
You are customer support AI assistant based for a company, your role is to make sure that customer reviews/comments and give befitting reponse
You are to use customer's review and the sentiment obtained from the user review provided below.
User review: {review}
sentiment: {sentiment}
"""

prompt = PromptTemplate(template=template, input_variables=["review", "sentiment"])

In [20]:
llm = GoogleGenerativeAI(model="gemini-pro")

In [21]:
chain = prompt | llm

In [22]:
# sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis", device="cuda:0") #using default model checkpoint with the T4 GPU as device.

# testing
sample_text = "I love this product, it works fine for me"
result = sentiment_analysis(sample_text)
print(result[0]["label"])


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


POSITIVE


In [23]:
def get_llm_result(query):
    sentiment = sentiment_analysis(query)[0]["label"]
    return chain.invoke({"review": query, "sentiment": sentiment})

In [24]:
test_query = "I love this product, it works fine for me"
get_llm_result(test_query)

"Thank you for your positive feedback! We're thrilled to hear that you're enjoying our product. We strive to provide our customers with high-quality products that meet their needs, and we're happy to know that we've succeeded in your case. If you have any further questions or comments, please don't hesitate to reach out."

In [26]:
# web app
def chatbot_response(user_input):
    return get_llm_result(user_input)

with gr.Blocks() as demo:
    gr.Markdown("# Sentiment based customer support Chatbot")

    input_textbox = gr.Textbox(label="Type in your review", placeholder="Enter your review message...")

    submit_button = gr.Button("Send review")
    output_textbox = gr.Textbox(label="Chatbot's Response")

    submit_button.click(fn=chatbot_response, inputs=input_textbox, outputs=output_textbox)

# Launch the Gradio interface
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d9375db6853d8893e9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
